# [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215v3)
- 그 유명한 seq2seq
- 어떤 내용일까. 간략하게 요약.

## Abstract
- 2개의 깊은(multi-layered) LSTM을 어떻게 사용하는지 보여준다.
    - 하나는 입력 문장을 고정된 차원의 벡터로 만들고(Encoder)
    - 다른 하나는 그 벡터로부터 target sequence를 출력한다.(Decoder)
---
- LSTM은 긴 문장을 학습하는데 어려움이 없고 민감한(sensible) 구나 문장 표현을 학습할 수 있다.
- 본 논문에서는 입력 문장의 단어 순서를 뒤집어서 LSTM의 성능을 markedly(눈에 띄게)향상시킬 수 있는 새로운 방법을 제시한다.
    - 단어 순서를 뒤집는 것이 더 좋은 명확한(정확한) 이유는 보일 수 없지만 그렇게 하는것이 source와 target의 단기의존성을 많이 제공하여, 최적화 문제를 더 간단하게 만들었다고 설명할 수 있다.

## Introduction
- DNN(Dense Neural Networks)의 장점
    1. 병렬 계산 가능(parallel computation)
    2. 복잡한 계산 가능(intricate computation)
    3. 역전파(labeling된 데이터가 있을 때의 supervised backpropagation)
---
- DNN의 단점
    1. Input과 target이 고정된 차원의 벡터로 인코딩 될 수 있는 문제만 사용 가능.
    - 하지만 현실에서는 sequence의 길이를 예측할 수 없음.
    - 그래서 **domain independent**한 seq2seq가 더 유용할 것이다.
---
#### 핵심 아이디어
1. 하나의 LSTM이 input sequence를 읽고 고정된 차원의 벡터 표현으로 바꾼다.
2. 다른 LSTM이 그 벡터 표현으로부터 output sentence를 추출(extract)한다.
- LSTM은 다른 시계열 모델보다 장기의존관계를 학습할 능력이 있기 때문에 사용하였다.
---
#### key technical contributions
- 입력 문장(source sentence)의 단어들의 순서를 뒤집음으로써 최적화 문제를 간단히 하여 학습을 용이하게 하였다.
- LSTM은 다양한 길이의 입력 문장을 고정된 차원의 벡터 표현으로 바꿀 수 있다는 점을 활용하였다.

## 2. The model
- input과 output의 길이가 다른 경우 어떻게 RNN을 적용할 수 있나?<br><br>
바로 2개의 RNN을 사용함으로써 그 문제를 해결할 수 있다.<br><br>
- 이러한 접근법(Encoder & Decoder)은 이미 전에 수행되었음.
---
#### 이 논문이 이전의 연구와 다른 점은?
1. 2개의 다른 LSTM을 사용하였다.
2. Deep LSTM(여러 층으로 구성된 LSTM) shallow LSTM보다 더 좋은 성능을 보이는 것을 확인하여, deep LSTM을 사용하였다.
3. 입력 문장의 단어 순서를 뒤집었다.

## 3. Experiments
- WMT'14 English to French MT task에 2가지 방법으로 적용하였다.
    1. SMT system에 대한 참고(reference)없이 바로 입력 문장을 해석.
    2. SMT baseline의 n-best lists를 rescore하였다.
        - 이건 무슨 뜻인지 정확히 모르겠음.
---
### 3-1. Dataset details
- 348M 개의 불어 단어와 304M개의 영어단어를 포함하는 12M개의 문장을 학습 데이터로 사용하였다. 
    - 이런 학습 데이터셋을 선택한 이유: n-best list & public availability
- 전형적인 neural language model은 각 단어의 벡터 표현에 의존하는 반면, 우리의 모델은 두 언어의 고정된 단어를 사용하였다.(고정된 단어의 개수)
    - 16만개의 source language 단어와 8만개의 target language 단어.
    - OOV 단어는 <"UNK"> token으로 대체됨.
---
### 3-2. Decoding and Rescoring
- source sentence가 주어졌을 때 정확한 해석 T에 대한 로그 가능도를 '최대화'하는 방식으로 학습을 진행하였다. 
    - 따라서, 목적 함수(objective function): ${1 \over |{\bf S}|} \sum_{(T,S) \in {\bf S}} \log p(T|S)$
        - $\bf S$는 훈련집합(training set)이다.
    - 훈련 1 EPOCH 종료 --> 다음 식에 따라LSTM에서 가장 그럴싸한(가능성 높은?) 해석을 출력한다. $\rightarrow \hat T = \arg \max_Tp(T|S)$
---
#### 어떻게 가장 그럴싸한 해석(the most likely translation)을 찾을까?
- **Beam Search**를 사용한다. 
- ※참고※ Beam Search에 대한 간단한 설명
    - Greedy searching을 보완한 기법.  누적확률분포를 고려하여  미리 설정한 beam size(k)만큼 선택한다. 자세한 것은 구글링,,,
- 이 논문에서는 beam size가 1일때도 seq2seq 모델은 잘 작동하며, beam size가 2일때 높은 성능을 보인다.(사실 12일 때 가장 높은 성능이지만, 연산량을 고려했을 때, 2로 하는것이 훨씬 효율적이라서 provide most of the benefits of beam search라고 적어 놓은 것 같음)
- baseline system에 의해 생성된 [1000-best lists를 rescore 한다]().
    - LSTM이 산출한 모든 가설(hypothesis)에 대한 로그 가능도를 계산하고 단순평균?(even average)을 계산한다.
        - even average는 무엇으로 해석,,?
---

### 3-3. Reversing the Source Senetences
- sourece 문장의 단어 순서가 뒤바뀌면(reversed) LSTM 훨씬 더 잘 학습한다는 것을 확인하였다.
    - test data에 대한 perplexity가 5.8에서 4.7로 감소하였고
    - BLEU score가 25.9에서 30.6으로 증가하였다.
- 본 논문에서는 이러한 현상에 대해 완벽한 설명은 제공하지 못한다. **'introduction of many short term dependencies'** 때문일 것이라고 생각한다.
    - source가 target으로 해석될 때, target문장에 대응하는 source 문장의 단어들은 target과 주로 멀리 떨어져있게 된다. 이것은 결과적으로 **['large minimal time lag'](https://proceedings.neurips.cc/paper/1996/file/a4d2f0d23dcc84ce983ff9157f8b7f88-Paper.pdf)**문제를 일으킨다.
    - Reversing해도 대응하는 단어들의 평균 거리는 바뀌지 않는다.
    - 하지만 첫 몇개의 source 단어는 target 단어와 굉장히 가까워지고 'minimal time lag'문제가 급격하게 줄어든다.
        - 그러므로 역전파를 통한 'establishing communication'이 더 용이하다.(source와 target사이)
    - 결과적으로 Reversing하는 것은 긴 문장을 학습하는데 효과적이다.(better memory utilization) 
---

### 3-4. Training details
- 1000개의 hidden cell(임베딩 차원도 1000)
- 4개의 LSTM층 사용 
    - Deep LSTM이 shallow보다 더 좋은 성능.
    - 하나 추가할 때마다 perlexity 10%정도 감소.
- input vocab은 16만개, output vocab은 8만개(고정)
- 결과적으로 384M개의 학습 파라미터
    - 64M개의 순환연결(32M Encoder, 32M Decoder)
1. 가중치 초기화: LSTM의 가중치를 -0.08~0.08사이의 균등 분포에 따라 초기화
2. momentum 없는 SGD 사용. 학습률은 처음에 0.7로 고정. 5 Epoch 이후부터는 epoch절반마다 학습률을 반으로 줄였다.
3. 배치 사이즈: 128. Gradient를 배치사이즈로 나눠주었음.
4. LSTM에도 가중치 폭발 문제가 여전히 존재했기 때문에, 임계값을 넘을 시 scaling 해주었다. 
    - l2 norm을 계산하였고, 임계값(5)을 넘으면 $g = {5g \over s}, \space (s = ||g||_2)$로 scaling 해주었다.
5. 대부분의 문장은 짧고 긴 문장은 거의 없었다. 미니배치가 랜덤으로 선택되어서, 많은 minibatch는 연산량을 낭비했음. 따라서 모든 minibatch의 문장들이 같은 길이를 가지도록 하였고 결과적으로 2배의 속도를 얻었다.
---

### 3-5. Parallelization
- DNN의 장점 중 하나.
- 하나의 GPU는 초당 1,700개 단어 연산 
    - 너무 느려서 8개의 GPU를 사용하였다.
        - 4개는 LSTM층 연산에 사용하고 남은 4개는 softmax를 병렬화 하는데 사용하였다.
        - 그래서 각 GPU는 1000 x 2000 크기의 행렬을 계산하였음.
- 결과적으로 배치사이즈 128에서 초당 영어와 불어 각각 6,300단어씩 연산을 수행할 수 있었다.
    - 학습하는데 10일이 걸렸다.
---

### 3-6. Experimental Results
- 번역의 질을 평가하는데 BLEU score를 사용하였다. 
- 결과표는 해당 논문에서 확인할 수 있다. 
- 1000 best list를 rescoring 하고 5개의 reversed LSTM을 사용했을 때, best WMT'14 score와 0.5 차이.(best WMT'14: 37, seq2seq:36.5)
---
### 3-7. Performance on long sentences
### 3-8. Model Analysis
- Test 결과 몇개 보여줌.
- PCA 차원축소로 유사한 문장들은 유사한 위치를 가진다는 것을 보여줌..?
---
## 4. Related work



---
## 5. Conclusion 
- Deep LSTM의 높은 성능 확인
- Reversed 훈련 방식의 높은 성능 확인.
- reversede dataset으로 학습된 LSTM이 긴 문장을 번역하는 데 큰 어려움이 없음을 확인.
- 직관적이고, 간단하고 비교적 덜 최적화된 접근방식이 SMT system을 능가했다는 것을 보여주었음.
    - 무슨 의미,,?

### [seq2seq 구현](https://github.com/farizrahman4u/seq2seq/blob/master/seq2seq/models.py)

In [2]:
#경로 바꿔주기
import os
path = 'C:/Users/alsrl/밑바닥부터 시작하는 딥러닝_2/ch07'
os.chdir(path)

In [3]:
#현재 경로 확인
os.getcwd()

'C:\\Users\\alsrl\\밑바닥부터 시작하는 딥러닝_2\\ch07'

In [4]:
import sys
sys.path.append('..')

### Framework를 사용하지 않는 naive한 구현(by 밑바닥2)
- 더하기 문제 구현하기
###### 1. Encoder

In [6]:
class Encoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size): #vocab_size: 어휘 수(문자의 종류)
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful = False) #LSTM이 계층 상태를 유지하지 않으므로 False 
        
        self.params = self.embed.params + self.lstm.params
        self.grads = self.embed.grads + self.lstm.grads
        self.hs = None
        
    def forward(self, xs):
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        self.hs = hs
        return hs[:, -1, :] #Encoder의 출력: 은닉상태에서 마지막 time step의 값
    
    def backward(self, dh):
        dhs = np.zeros_like(self.hs)
        dhs[:, -1, :] = dh 
        
        dout = self.lstm.backward(dhs)
        dout = self.embed.backward(dout)
        return dout 

##### 코드 뜯어보기 ① Encoder
```python
class Encoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
```
- vocab_size: 문자(단어)의 종류. 
- wordvec_size: embedding된 문자(단어)벡터의 차원 수.
- hidden_size: LSTM 계층에서 은닉 상태 벡터의 차원 수
---
```python
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
```
- 가중치 초기화 방식. 
    - 임베딩 할 때 사용되는 가중치들은 100으로 정규분포를 100으로 나누어서 초기화
    - 입력 가중치는 임베딩 벡터의 우너 수의 제곱근으로 나누어 초기화.
    - 은닉 상태의 가중치는 은닉 벡터 차원 수의 제곱근으로 나누어 초기화.
---
```python
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful = False) #LSTM이 계층 상태를 유지하지 않으므로 False 
```
- 각 층을 만들어준다. 
    - TimeEmbedding과 TimeLSTM은 책에서 사용하는 임베딩, LSTM 층의 이름
        - stateful = False --> 은닉상태 유지할 필요 없기 때문에
---
```python
        self.params = self.embed.params + self.lstm.params
        self.grads = self.embed.grads + self.lstm.grads
        self.hs = None
```
- 리스트 들의 합이다.(self.embed.params는 list형태 self.lstm.params 또한 리스트 형태임)
    - grads 또한 전부 기울기 형태로, 다 더해서 하나의 params list와 grads list에 저장해둔다.
---
```python
    def forward(self, xs):
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        self.hs = hs
        return hs[:, -1, :]
```
- forward는 xs를 입력으로 받아 embeding 층과 lstm층을 차례로 통과시킨다
    - lstm층을 통과한 hs는 self.hs에 저장된다. 
    - Encoder는 입력을 하나의 고정된 벡터로 출력하는 것을 목표로 하기 때문에 forward의 출력은 hs(은닉층)의 마지막 time step 값이다.
---
```python
    def backward(self, dh):
        dhs = np.zeros_like(self.hs)
        dhs[:, -1, :] = dh 
        
        dout = self.lstm.backward(dhs)
        dout = self.embed.backward(dout)
        return dout 
```
- backward때는 입력으로 받은(손실함수로 계산된 값) dh를, dhs[:, -1, :]에 저장한다.
    - dhs는 hs와 같은 shape을 가진 영텐서임.
    - backward의 결과값 dout를 return한다.

###### 2. Decoder

In [7]:
class Decoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size): #vocab_size: 어휘 수(문자의 종류)
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b =  np.zeros(V).astype('f')
        
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful = False) 
        self.affine = TimeAffine(affine_W, affine_b)
        
        self.params, self.grads = [], []
        for layer in (self.embed, self.lstm, self.affine):
            self.params += layer.params
            self.grads += layer.grads
        
    def forward(self, xs, h):
        self.lstm.set_state(h)
        
        out = self.embed.forward(xs)
        out = self.lstm.forward(out)
        score = self.affine.forward(out)
        return score
    
    def backward(self, dscore):
        dout = self.affine.backward(dscore)
        dout = self.lstm.backward(dout)
        dout = self.embed.backward(dout)
        dh = self.lstm.dh
        return dh
    
    def generate(self, h, start_id, sample_size):
        sampled = []
        sample_id = start_id
        self.lstm.get_state(h)
        
        for _ in range(sample_size):
            x = np.array(sample_id).reshape((1,1))
            out = self.embed.forward(x)
            out = self.lstm.forward(out)
            score = self.affine.forward(out)
            
            sample_id = np.argmax(score.flatten())
            sampled.append(int(sample_id))
            
        return sampled

###### 코드 뜯어보기 ② Decoder
- Encoder와 거의 유사함
```python
        self.affine = TimeAffine(affine_W, affine_b)
```
- Affine연결 층이 있다는 것을 제외하면 거의 비슷.
--- 
- forward(순전파)에서는 xs와 은닉벡터(h) 2개를 입력으로 받는다.
---
```python
    def generate(self, h, start_id, sample_size):
        sampled = []
        sample_id = start_id
        self.lstm.get_state(h)
        
        for _ in range(sample_size):
            x = np.array(sample_id).reshape((1,1))
            out = self.embed.forward(x)
            out = self.lstm.forward(out)
            score = self.affine.forward(out)
            
            sample_id = np.argmax(score.flatten())
            sampled.append(int(sample_id))
            
        return sampled
```
- 입력을 받으면 Affine 층의 출려에서 가장 큰 id를 선택한다.

##### 3. Encoder/Decoder 연결

In [5]:
from common.time_layers import *
from common.base_model import BaseModel

class Seq2seq(BaseModel):
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        self.encoder = Encoder(V, D, H)
        self.decoder = Decoder(V, D, H)
        self.softmax = TimeSoftmaxWithLoss()
        
        self.params = self.encoder.params + self.decoder.params
        self.grads = self.encoder.grads + self.decoder.grads
        
    def forward(self, xs, ts):
        decoder_xs, decoder_ts = ts[:, :-1], ts[:, 1:]
        h = self.encoder.forward(xs)
        score = self.decoder.forward(decoder_xs, h)
        loss = self.softmax.forward(score, decoder_ts)
        return loss
    
    def backward(self, dout = 1):
        dout = self.softmax.backward(dout)
        dh = self.decoder.backward(dout)
        dout = self.encoder.backward(dh)
        return dout
    
    def generate(self, xs, start_id, sample_size):
        h = self.encoder.forward(xs)
        sampled = self.decoder.generat(h, start_id, sample_size)
        return sampled

#### Pytorch를 이용한 Encoder & Decoder 구현

### 1. Encoder
- 문자열을 받아서 고정된 길이의 벡터로 출력
- 모든 층은 LSTM으로 구성된다.


In [1]:
class LSTMDecoderCell(ExtendedRNNCell):

ModuleNotFoundError: No module named 'recurrentshop'

### 2. Decoder 

### 3. Seq2Seq 
- Encoder과 Decoder를 연결